In [5]:
n=3

In [7]:
G = SymmetricGroup(n)

In [7]:
SGA = G.algebra(QQ)

In [8]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1 -1/2 -1/2  1/2  1/2   -1]
[   0 -3/4  3/4  3/4 -3/4    0]
[   0    1   -1    1   -1    0]
[   1 -1/2 -1/2 -1/2 -1/2    1]
[   1    1    1   -1   -1   -1]

In [93]:
#is the matrix unitary?
#A*A^T, the Gram matrix, is diagonal, but not the identity. so A is not unitary.

In [9]:
print((A*A.transpose()).str())

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]


In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick: 
#define a new inner product <x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#compute an orthonormal basis using this inner product

In [434]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,unitary=False):
    n = sum(partition)
    G = SymmetricGroup(n)
    specht_module = G.algebra(QQ).specht_module(partition)
    rho = specht_module.representation_matrix
    d_part = specht_module.dimension()
    if unitary:
        M = unitary_change_of_basis(partition,base_ring=QQbar)
        return sqrt(d_part/G.order())*sum(f(g)*M.inverse()*rho(g)*M for g in G)
    else:
        return sum(f(g)*rho(g) for g in G)

In [178]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
def delta(s):
    return lambda t: 1 if t == s else 0

In [411]:
#for each basis element g \in G
#compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
#make the Fourier coefficients into a list via .list()
#flatten the list of lists to get a vector \hat{g}
#put the vectors into a list and take the transpose to get the DFT
from sage.misc.flatten import flatten
def unitary_dft(n):
    return matrix([flatten([hat(delta(g),partition,unitary=True).list() for partition in Partitions(n)]) for g in G]).transpose()

In [378]:
#define the G-averaged inner product given a representation
#<x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#where x, y are coordinate vectors for elements in V_\rho
def inv_inner_product(partition,x,y):
    n = sum(partition)
    G = SymmetricGroup(n)
    rho = G.algebra(QQ).specht_module(partition).representation_matrix
    return (1/G.order())*sum((rho(g)*vector(x))*(rho(g)*vector(y)) for g in G)

In [379]:
#define the projection of the vector v onto vector u
def proj(u,v):
    return ((vector(v)*vector(u))/(vector(u)*vector(u)))*vector(u)

In [391]:
#define Gram-Schmidt orthonormalization process that works in the symbolic ring
#assume A = [v_1, v_2, ... , v_k] and each v_j is a column vector
#NOTE: GS, M = A.gram_schmidt(orthonormal=True) works for some rings, but not \QQ or exact rings
def gram_schmidt_SR(A,partition,orthonormal=False):
    v = A.transpose()
    u = []
    e = []
    for k in range(A.ncols()):
        u_k = v[k] - sum(proj(u[j],v[k]) for j in range(k))
        u.append(u_k)
        e.append(u_k/(u_k.norm()))
    if orthonormal:
        return matrix(e).transpose()
    else:
        return matrix(u).transpose()

In [458]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(partition,base_ring=QQ):
    A = gram_matrix(partition,base_ring)
    #orthonormal_vecs = gram_schmidt_SR(A,partition,orthonormal=True)
    orthonormal_vecs, M = A.gram_schmidt(orthonormal=True)
    return M

In [416]:
#define the inner product matrix A = [<e_i,e_j>_\rho] for some ordered basis of {e_i} of V_\rho
def gram_matrix(partition,base_ring=QQ):
    specht_module = G.algebra(base_ring).specht_module(partition)
    d_part = specht_module.dimension()
    e = lambda i: [1 if i==j else 0 for j in range(d_part)]
    return matrix(base_ring,[[inv_inner_product(partition,e(i),e(j)) for j in range(d_part)] for i in range(d_part)])

In [469]:
#ISSUE: the change-of-basis matrix is not making the representations unitary
#DESCRIPTION: both Gram-Schmidt methods are failing.
#the change-of-basis taking you from the orthonormal basis to the standard basis should be what's required.

In [61]:
partition = Partitions(n)[1]

In [501]:
specht_module = G.algebra(QQ).specht_module(partition)

In [131]:
rho = specht_module.representation_matrix

In [105]:
d_part = specht_module.dimension()

In [518]:
A_2 = gram_matrix(partition,base_ring=QQ)

In [519]:
A_1 = gram_matrix(partition,base_ring=QQbar)

In [520]:
#gram_schmidt uses QR to actually compute the orthonormal vectors and coefficient matrix
#the built-in method is also not working
M_1 = unitary_change_of_basis(partition,base_ring=QQbar); M_1

[ 1.490711984999860?                   0]
[-1.192569587999888? 0.8944271909999159?]

In [510]:
orthonormal_vecs = gram_schmidt_SR(A,partition,orthonormal=True); orthonormal_vecs

[ 2/5*sqrt(5)    sqrt(1/5)]
[-1/5*sqrt(5)  2*sqrt(1/5)]

In [522]:
M_2 = A_2*orthonormal_vecs.inverse(); M_2

[ 1.490711984999860?                   0]
[-1.192569587999888? 0.8944271909999159?]

In [517]:
orthonormal_vecs, M = A_2.gram_schmidt(orthonormal=True); M*orthonormal_vecs - A

[0.?e-18 0.?e-18]
[0.?e-17 0.?e-17]

In [504]:
#check that inner products of columns are really zero
orthonormal_vecs.transpose()[0]*orthonormal_vecs.transpose()[1]

0

In [503]:
#check that \rho(g) is unitary w.r.t. invariant inner product
x = vector([1,2]); y = vector([2,3])
all(inv_inner_product(partition,rho(g)*x,rho(g)*y) == inv_inner_product(partition,x,y) for g in G)

True

In [523]:
U = M.inverse()*rho(G[5])*M; U*U.transpose()

[3.600000000000000? 7.800000000000000?]
[7.800000000000000? 17.17777777777778?]

In [493]:
#check that the DFT is unitary
unitary_dft(3)*unitary_dft(3).transpose()

[                                        1                                         0                                         0                                         0                                         0                                         0]
[                                        0                                    244/75                                    -26/25                                  1586/225                                   -169/75 sqrt(1/3)*sqrt(1/6) - sqrt(1/3)*sqrt(1/6)]
[                                        0                                    -26/25                                     12/25                                   -169/75                                     26/25                                         0]
[                                        0                                  1586/225                                   -169/75                                 14884/675                                 -1586/225                            

In [58]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [92]:
eigs = A.eigenvalues(); eigs

[-1.927382440195288?, 2.121715010353017?, -1.174423554578382? - 1.623003052198150?*I, -1.174423554578382? + 1.623003052198150?*I, 1.452257269499517? - 1.086817272076326?*I, 1.452257269499517? + 1.086817272076326?*I]

In [93]:
[abs(eig) for eig in eigs]

[1.927382440195288?,
 2.121715010353017?,
 2.003349593304431?,
 2.003349593304431?,
 1.813897174510622?,
 1.813897174510622?]

In [121]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))

[2.44948974+0.j 2.44948974+0.j 2.        +0.j 1.73205081+0.j
 1.73205081+0.j 1.5       +0.j]
[2.44948974 1.73205081 1.5        2.         1.73205081 2.44948974]
